In [1]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib

In [5]:
df = pd.read_csv(r'C:\Users\LENOVO\Desktop\Desk Files/IMDBMovie.csv')
df

,ID,Title,Genre,Director,Year,Runtime_minutes,Rating,Votes,Revenue_millions
0,1,Guardians of the Galaxy,Action,James Gunn,2014,121,8.1,757074,333.13
1,2,Prometheus,Adventure,Ridley Scott,2012,124,7.0,485820,126.46
2,3,Split,Horror,M. Night Shyamalan,2016,117,7.3,157606,138.12
3,4,Sing,Animation,Christophe Lourdelet,2016,108,7.2,60545,270.32
4,5,Suicide Squad,Action,David Ayer,2016,123,6.2,393727,325.02
...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,Crime,Billy Ray,2015,111,6.2,27585,NaN
996,997,Hostel: Part II,Horror,Eli Roth,2007,94,5.5,73152,17.54
997,998,Step Up 2: The Streets,Drama,Jon M. Chu,2008,98,6.2,70699,58.01
998,999,Search Party,Adventure,Scot Armstrong,2014,93,5.6,4881,NaN


In [9]:
len(df)

1000

What is the 75th percentile of rating in the IMDB dataset?

In [8]:
rating_75th = df['Rating'].quantile(0.75)
rating_75th

7.4

How many NA values are there in the field ‘Revenue’?

In [13]:
df['Revenue_millions'].isnull().sum()

128

Data slicing:

We need to slice and dice the data to get a fair understanding of top earning movies, underrated movies and some additional statistics such as yearly revenue and average ratings

How many movies have revenue higher than 75 million?

In [17]:
movies_with_higher_revenue_75 = len(df[df['Revenue_millions']>75])
movies_with_higher_revenue_75

318

How many movies have revenue greater than 50 million but rating less than 7?

In [24]:
count_movies = len(df[(df['Revenue_millions']>50) & (df['Rating']<7)])
count_movies

211

What is the total revenue generated by movies in the year 2015?

In [26]:
revenue_generated_2015 = df[df['Year']==2015]['Revenue_millions'].sum()
round(revenue_generated_2015, 2)

8854.12

What is the average rating for the genre adventure in the year 2015?

In [29]:
average_rtg_adventure = df[(df['Year']==2015) & (df['Genre']=='Adventure')]['Rating'].mean()
average_rtg_adventure

6.8

What is the average duration of movies in rows 75 to 150? Please note that the rows in python start from 0.

In [31]:
average_duration_movies = df.iloc[75:150]['Runtime_minutes'].mean()
round(average_duration_movies, 2)

127.61

Which year generated the highest revenue?

In [32]:
Highest_revenue_year =  df.groupby('Year')['Revenue_millions'].sum()
Highest_revenue_year.idxmax()

2016

What is the maximum revenue out of (10,20,30,40,50) rows?

In [34]:
max_revenue = df.iloc[10:51]['Revenue_millions'].max()
max_revenue

936.63

How many movies with the genres ‘Adventure’, ‘Action’, ‘Horror’, and ‘Crime’ exist in the IMDB dataset?

In [50]:
n_movies_per_genre = df["Genre"].str.split(",").apply(lambda x: x.count(Genre) for Genre in Genres)

n_movies = n_movies_per_genre.sum()
n_movies.sum()

485

Create a genre-level report with metrics average rating, the average number of votes, and average revenue. What is the average rating of the ‘Horror’ genre? (Round to 2 decimal places)

In [52]:
genre_report = df.groupby('Genre').agg({
    'Rating': 'mean',
    'Votes' : 'mean',
    'Revenue_millions' : 'mean'}).reset_index()

genre_report.columns = ['Genre', 'Average Rating', 'Average votes', 'Average Revenue']

round(genre_report, 2)

,Genre,Average Rating,Average votes,Average Revenue
0,Action,6.59,235948.58,122.09
1,Adventure,6.91,225669.32,113.45
2,Animation,7.32,208159.43,191.22
3,Biography,7.32,153060.72,55.95
4,Comedy,6.49,115640.19,51.58
5,Crime,6.81,150124.96,41.04
6,Drama,6.95,129445.13,35.87
7,Fantasy,5.85,106723.00,63.11
8,Horror,5.87,79435.41,39.95
9,Mystery,6.88,218672.00,64.38


Create a report to showcase the revenue of each movie, as % revenue concerning the total revenue of the respective genre and year of the movie.
What is the  % revenue of the movie ‘Split’ in its respective genre and year?


In [57]:
genre_year_total_revenue = df.groupby(['Genre','Year'])['Revenue_millions'].transform('sum')

split_revenue = df[(df['Title']=='Split')]['Revenue_millions'].values[0]

percentage_revenue_split = (split_revenue * 100)/genre_year_total_revenue[df['Title']=='Split'].values[0]

round(percentage_revenue_split, 2)

29.42

What is the average ‘Votes_norm’ ?

In [58]:
max_votes = df['Votes'].max()
min_votes = df['Votes'].min()

def normalize_votes(votes):
    return ((votes - min_votes) * 10)/(max_votes - min_votes)

df['votes_norm'] = df['Votes'].apply(normalize_votes)

average_votes_norm = df['votes_norm'].mean().round(2)

average_votes_norm

0.95

What is the highest ‘Total_rating’ ?

In [60]:
def Cal_total_rtg(row):
    return row['Rating'] + row['votes_norm']

df['Total_rating'] = df.apply(Cal_total_rtg, axis=1)

highest_total_rating = df['Total_rating'].max()
highest_total_rating

19.0

Create a new column ‘Revenue_bins’ so that ‘Revenue_millions’ are categorized in buckets 0-50, 51-100, 101-150, and 150+. Which bucket has the highest number of movies?

In [62]:
bins = [0, 50, 100, 150, float('inf')]
labels = ['0-50', '51-100', '101-150', '150+']

df['Revenue_bins'] = pd.cut(df['Revenue_millions'], bins=bins, labels=labels)

highest_bucket = df['Revenue_bins'].value_counts().idxmax()

highest_bucket

'0-50'

*** END ***